<a href="https://colab.research.google.com/github/Gail529/Mental-health/blob/main/lstm_model_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import gensim
import os
import tweepy as tw
import streamlit as st
 
 
 
#tweet preprocessing 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from keras import backend as K
from tensorflow.keras.models import Model, load_model
from nltk.tokenize import word_tokenize,sent_tokenize,TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
 
 
 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
 
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
data=pd.read_csv('/content/tweet_emotion_scores.csv')

In [ ]:
data=pd.read_csv('/content/Mental_health.csv')

In [ ]:
data.head()


,Unnamed: 0,tweets,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,afinn_score,positive_vader,negative_vader,neutral_vader,emotion_score,labels
0,0,people can also take advantage of your phone a...,2,2,1,1,1,1,1,1,1,1,2.0,0.090,0.101,0.809,0.283441,2
1,1,christmas is a good time to start practicing g...,3,0,0,3,0,1,3,0,1,1,3.0,0.388,0.000,0.612,0.110710,1
2,2,driving in the midst of depression mindwand...,1,2,0,1,0,1,0,1,0,0,0.0,0.087,0.315,0.598,0.171658,1
3,3,may the prince of peace give u all peace of ou...,4,0,0,2,0,0,2,0,0,2,4.0,0.292,0.000,0.708,0.075926,1
4,4,anyone else just pretending they re ok sadtw...,0,2,1,0,0,0,0,1,0,0,-3.0,0.220,0.415,0.366,0.184060,1


In [ ]:
data.loc[(data['Positive'] < data['Negative']) & (data['Sadness'] >= 1) , 'score']= '1'


In [ ]:
#lowercasing and url,punctuations and numbers removal,
def Lowercasing(words):
    string=re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "",str(words))
    word=string.lower()
    return word

#Tokenization and (@)handle extraction
def Tokenization(tweet):
    tokenizer=TweetTokenizer(strip_handles=True)
    tokens = tokenizer.tokenize(tweet)
    return tokens

#punctuations
def Punctuation_removal(tokens):
    words=[ word for word in tokens if word.isalnum()]
    return words

#stemming
def stemming(text):
    stemmer=PorterStemmer()
    for  word in text:
        stemmed_words=stemmer.stem(word)
        return stemmed_words

#stopword_removal
def remove_stopwords(words):
    stop_words=set(stopwords.words("english")) 
    result=[word for word in words if word not in stop_words ]
    return result


#lemmatization
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    lemmatized_phrase=[]
    for word in text:
        lemmatized_word=lemmatizer.lemmatize(word)
        lemmatized_phrase.append(lemmatized_word)
    return lemmatized_phrase
 

In [ ]:
tokenized_tweets=[]
final_tweet=[]

#tokenization , (@)handle removal and lemmatization
for i in range(0,2000):
    tweet=tweets[i]
    tweet_tokens=Tokenization(tweet)
    lemmatized_tweet=lemmatization(tweet_tokens)#lemmatization
    tokenized_tweets.append(lemmatized_tweet)

#lowercasing url extraction and punctuation removal
for i in tokenized_tweets:
    lowercased_string=Lowercasing(i)#lowercasing and removing numbers
    final_tweet.append(lowercased_string)

In [ ]:
del data['Unnamed: 0']
del data['user']
del data['location']
del data['text']
data.head()

,tweets,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,score
0,people can also take advantage of your phone a...,2,2,1,1,1,1,1,1,1,1,0
1,christmas is a good time to start practicing g...,3,0,0,3,0,1,3,0,1,1,0
2,driving in the midst of depression mindwand...,1,2,0,1,0,1,0,1,0,0,1
3,may the prince of peace give u all peace of ou...,4,0,0,2,0,0,2,0,0,2,0
4,anyone else just pretending they re ok sadtw...,0,2,1,0,0,0,0,1,0,0,1


In [ ]:
tweets=data['tweets'].values.tolist()
len(tweets)

1999

Tokenizing the tweet

In [ ]:
tweet_list=[]
for tweet in tweets:
    tokens =  word_tokenize(tweet)
    #stop_words=set(stopwords.words("english")) 
    #word=[word for word in tokens if word not in stop_words]
    tweet_list.append(tokens)

    

Training the word2vec Model

In [ ]:
EMBEDDING_DIM=100
model = gensim.models.Word2Vec(sentences=tweet_list,size=EMBEDDING_DIM,window=5,workers=4,min_count=1)
words = list(model.wv.vocab)
print('Vocabulary size: %d',len(words))
print(words)

Vocabulary size: %d 4908
['people', 'can', 'also', 'take', 'advantage', 'of', 'your', 'phone', 'addiction', 'before', 'you', 'dont', 'treat', 'it', 'right', 'away', 'download', 'our', 'app', 'now', 'christmas', 'is', 'a', 'good', 'time', 'to', 'start', 'practicing', 'gratitude', 'check', 'out', 'how', 'will', 'change', 'life', 'driving', 'in', 'the', 'midst', 'depression', 'mindwandering', 'cause', 'higher', 'risk', 'be', 'careful', 'pl', 'may', 'prince', 'peace', 'give', 'u', 'all', 'mind', 'body', 'soul', 'from', 'today', 'going', 'forward', 'prayerforlove', 'anyone', 'else', 'just', 'pretending', 'they', 're', 'ok', 'sadtweets', 'sad', 'at', 'this', 'point', 'ain', 't', 'no', 'need', 'friend', 'for', 'help', 'trend', 'therefore', 'wisdom', 'gods', 'word', 'bring', 'christ', 'jesus', 'heart', 'without', 'delay', 'adult', 'there', 'still', 'struggling', 'with', 'abandonment', 'issue', 'caused', 'by', 'parent', 'divorce', 'know', 'that', 'yourself', 'gift', 'self', 'love', 'holiday', '

In [ ]:
model.wv.most_similar('depression')

[('diagnosed', 0.9967317581176758),
 ('wa', 0.9963631629943848),
 ('anxiety', 0.9959952235221863),
 ('mentalhealth', 0.9959667921066284),
 ('year', 0.9958199262619019),
 ('get', 0.9953647255897522),
 ('people', 0.9949799180030823),
 ('health', 0.994754433631897),
 ('like', 0.994587779045105),
 ('time', 0.9944572448730469)]

Saving the word2vec model

In [ ]:
filename = 'tweets_embedding.txt'
model.wv.save_word2vec_format(filename,binary=False)

Preparing Train and Test Data


In [ ]:
def prep_tokenizer(tweet_list):
    #instantiating the tokenizing object -creating a unique word dictionary ,where unique words are rep. as sentences
    tokenizer = Tokenizer()
    #fitting it into the text
    tokenizer.fit_on_texts(tweet_list)
    #saving the tokenizing object
    with open(‘tokenizer.pickle’, ‘wb’) as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(“Tokeniser saved”)
 
def prepare_data(tweet_list,tokenizer):
   #creating a list of tokens for each tweet (A list of lists)
   sequences = tokenizer.texts_to_sequences(tweet_list)
   word_index = tokenizer.word_index
   print(‘Vocabulary size:’, len(word_index))
   max_length = max([len(s.split()) for s in tweets])
   tweet_pad =pad_sequences(sequences,maxlen=max_length)
   return tweet_pad

 

In [ ]:
prep_tokenizer(tweet_list)
with open(‘tokenizer.pickle’, ‘rb’) as handle:
   tokenizer = pickle.load(handle)

tweet_pad = prepare_data(tweet_list,tokenizer)

In [ ]:
emotion_score=data['labels'].values

VALIDATION_SPLIT = 0.2
indices = np.arange(tweet_pad.shape[0])
np.random.shuffle(indices)
tweet_pad = tweet_pad[indices]
emotion_score = emotion_score[indices]

num_validation_samples = int(VALIDATION_SPLIT * tweet_pad.shape[0])

x_train_pad = tweet_pad[:-num_validation_samples]
y_train = emotion_score[:-num_validation_samples]
x_test_pad = tweet_pad[-num_validation_samples:]
y_test = emotion_score[-num_validation_samples:]

In [ ]:
x_train_pad = np.asarray(x_train_pad).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
x_test_pad = np.asarray(x_test_pad).astype(np.float32)
y_test= np.asarray(y_test).astype(np.float32)


print(len(x_test_pad))
print(len(x_train_pad))
print(len(y_train))
print(len(y_test))

Creating the Embedding layer

In [ ]:
import os
embeddings_index = {}
f = open(os.path.join('','tweets_embedding.txt'),encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()    

Embedding Matrix

In [ ]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))

for word,i in word_index.items():
    if i > num_words:
        continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
print(num_words)

4909


In [ ]:
model = Sequential()
embedding_layer = Embedding(num_words ,EMBEDDING_DIM)                         
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(1, activation="tanh"))
model.add(Dense(6,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam' , metrics=['accuracy'])


Model Training Finally....

In [ ]:
model.fit(x_train_pad,y_train, batch_size=10, epochs=20 , validation_data=(x_test_pad,y_test),verbose=2)

Epoch 1/20
160/160 - 4s - loss: 1.6233 - accuracy: 0.5800 - val_loss: 1.4439 - val_accuracy: 0.5965
Epoch 2/20
160/160 - 2s - loss: 1.3227 - accuracy: 0.6206 - val_loss: 1.2390 - val_accuracy: 0.5965
Epoch 3/20
160/160 - 2s - loss: 1.1499 - accuracy: 0.6206 - val_loss: 1.1218 - val_accuracy: 0.5965
Epoch 4/20
160/160 - 2s - loss: 1.0346 - accuracy: 0.6206 - val_loss: 1.0521 - val_accuracy: 0.5965
Epoch 5/20
160/160 - 2s - loss: 0.9516 - accuracy: 0.6206 - val_loss: 1.0035 - val_accuracy: 0.5965
Epoch 6/20
160/160 - 2s - loss: 0.8879 - accuracy: 0.6281 - val_loss: 0.9612 - val_accuracy: 0.5965
Epoch 7/20
160/160 - 2s - loss: 0.8382 - accuracy: 0.6431 - val_loss: 0.9269 - val_accuracy: 0.5990
Epoch 8/20
160/160 - 2s - loss: 0.7963 - accuracy: 0.6619 - val_loss: 0.9003 - val_accuracy: 0.6015
Epoch 9/20
160/160 - 2s - loss: 0.7636 - accuracy: 0.6719 - val_loss: 0.8689 - val_accuracy: 0.6165
Epoch 10/20
160/160 - 2s - loss: 0.7334 - accuracy: 0.6769 - val_loss: 0.8524 - val_accuracy: 0.6140

In [ ]:
model.fit(x_train_pad,y_train, batch_size=20, epochs=20 , validation_data=(x_test_pad,y_test),verbose=2)
lstm_model.save('final_lstm.h5')


Epoch 1/20
80/80 - 2s - loss: 0.3579 - accuracy: 0.9175 - val_loss: 0.6641 - val_accuracy: 0.7619
Epoch 2/20
80/80 - 1s - loss: 0.3479 - accuracy: 0.9131 - val_loss: 0.6640 - val_accuracy: 0.7594
Epoch 3/20
80/80 - 1s - loss: 0.3379 - accuracy: 0.9131 - val_loss: 0.6626 - val_accuracy: 0.7519
Epoch 4/20
80/80 - 1s - loss: 0.3293 - accuracy: 0.9137 - val_loss: 0.6645 - val_accuracy: 0.7519
Epoch 5/20
80/80 - 1s - loss: 0.3215 - accuracy: 0.9131 - val_loss: 0.6683 - val_accuracy: 0.7494
Epoch 6/20
80/80 - 1s - loss: 0.3138 - accuracy: 0.9137 - val_loss: 0.6695 - val_accuracy: 0.7519
Epoch 7/20
80/80 - 1s - loss: 0.3062 - accuracy: 0.9150 - val_loss: 0.6715 - val_accuracy: 0.7519
Epoch 8/20
80/80 - 1s - loss: 0.3001 - accuracy: 0.9150 - val_loss: 0.6704 - val_accuracy: 0.7519
Epoch 9/20
80/80 - 1s - loss: 0.2954 - accuracy: 0.9150 - val_loss: 0.6746 - val_accuracy: 0.7544
Epoch 10/20
80/80 - 1s - loss: 0.2898 - accuracy: 0.9150 - val_loss: 0.6707 - val_accuracy: 0.7494
Epoch 11/20
80/80 -

In [ ]:
txt=["who new life would be so hard this fast"]
seq = tok_obj.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
print(np.argmax(pred))


1


In [ ]:
#Creating the tokenisation object
def tokenizer(tweet_list){
    tok_obj = Tokenizer()
    return (tok_obj.fit_on_texts(tweet_list))
    }

tok_obj=tokenizer(tweet_list)
sequences = tok_obj.texts_to_sequences(tweet_list)
word_index=tok_obj.word_index
print(len(words))
max_length = max([len(s.split()) for s in tweets])

tweet_pad =pad_sequences(sequences,maxlen=max_length)
emotion_score=data['labels'].values

print('Shape of tweet Tensor:',tweet_pad.shape)
print('Shape of emotion_score Tensor:',emotion_score.shape)



# Streamlit code


In [ ]:
MODEL_PATH = r'final_lstm.h5'
MAX_NB_WORDS = 100000 # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 200 # max length of each entry (sentence), including padding
VALIDATION_SPLIT = 0.2 # data for validation (not used in training)
EMBEDDING_DIM = 100
tokenizer_file = “tokenizer.pickle”

In [ ]:
with open(tokenizer_file, ‘rb’) as handle:
   tokenizer = pickle.load(handle)
@st.cache(allow_output_mutation=True)
def Load_model():
   model = load_model(MODEL_PATH)
   model._make_predict_function()
   model.summary() # included to make it visible when model is reloaded
   session = K.get_session()
   return model, session

In [ ]:
if predict_btt:
   clean_text = []
   K.set_session(session)
   i = basic_text_cleaning(sentence)
   clean_text.append(i)
   sequences = tokenizer.texts_to_sequences(clean_text)
   data = pad_sequences(sequences, padding = ‘post’, maxlen =  MAX_SEQUENCE_LENGTH)
   prediction = model.predict(data)